In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./medquad.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
https://huggingface.co/datasets/lavita/ChatDoctor-HealthCareMagic-100k/viewer/default/train?views%5B%5D=train

# DeepSeek R1 Model Fine-Tuning (LORA) with GPT-4 Dataset [Unsloth and OLLAMA]

## **Unsloth**

Unsloth appears to be a tool or framework designed for efficient fine-tuning of language models. From the context, it likely incorporates techniques like Low-Rank Adaptation (LoRA) and other efficiency optimizations to fine-tune large models, such as Llama derivatives, with minimal computational resources.

The key features of Unsloth as implied by your description might include:

1. **Efficient Fine-Tuning**: Instead of updating the entire model's weights, it leverages methods like LoRA, which fine-tune a smaller subset of the parameters, making it resource-efficient.

2. **Simplified Workflow**: The process of loading models, configuring parameters, and training appears streamlined, allowing developers to focus on specific customizations rather than managing complex infrastructure.

3. **Integration with Local Runtimes**: After fine-tuning, exporting to tools like Ollama for local deployment demonstrates its support for practical application.

## **DeepSeek-R1**

The DeepSeek-R1 is a versatile robot for exploration and inspection in tough environments. With AI, precise sensors, and multi-terrain mobility, it handles tasks like data collection, mapping, and monitoring. Customisable for search and rescue, inspections, or research, it ensures reliable performance in hazardous areas.

![](https://bsmedia.business-standard.com/_media/bs/img/article/2025-01/28/full/1738047877-0145.JPG?im=FeatureCrop,size=(826,465))

In [3]:
!pip install unsloth


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Found existing installation: unsloth 2025.3.17
Uninstalling unsloth-2025.3.17:
  Successfully uninstalled unsloth-2025.3.17
  Cloning https://github.com/unslothai/unsloth.git to c:\users\user\appdata\local\temp\pip-req-build-qujn90bl
  Resolved https://github.com/unslothai/unsloth.git to commit 65b8975c5fb65e6c08726f228877ba6b6601f2ba
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for unsloth: filename=unsloth-2025.3.17-py3-none-any.whl size=197137 sha256=c040c2b1d8717babe6964d6ec8e148cf7446a5e1ac5d712636bc5a168c4cfcf9
  Stored in directory: C:\Users\user\AppData\Local\Temp\pip-ephem-wheel-cache-hh7u4f3t\wheels\60\3e\1f\e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built uns

  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-qujn90bl'

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load the Model
Using Unsloth, you load the DeepSeek-R1 Distilled Llama-8B model, a smaller, faster version of the Llama model optimised for performance while retaining accuracy.
Along with the model, you also load its tokeniser. The tokeniser breaks down input text into smaller units (tokens) that the model can process.AttributeError

## Importance?
Loading the model and tokenizer is the foundation for fine-tuning since they define how text inputs are processed and predictions are generated.

In [6]:

from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

C:\Users\user\AppData\Local\Temp\ipykernel_10012\2543187387.py:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


UnicodeDecodeError: 'cp950' codec can't decode byte 0xcf in position 3539: illegal multibyte sequence

## Setup Parameter-Efficient Fine-Tuning (PEFT) 
The **FastLanguageModel.get_peft_model** function modifies a pre-trained language model to use PEFT techniques, which allow fine-tuning of the model using fewer resources and parameters.

The method introduces additional tunable parameters (like LoRA matrices) to specific layers of the model while freezing most of the original model weights.

***In this Project, we use LORA to fine-tune the DeepSeek R1 LLM.***

First, it takes the existing model. Then, it applies a technique called PEFT (Parameter-Efficient Fine-Tuning).  Think of it as adding small, adjustable "knobs" instead of changing the whole engine.

r=4 and lora_alpha=16 are just settings for how many "knobs" and how sensitive they are.  target_modules specifies where these knobs are attached – specifically, parts of the model that handle questions, keys, values, and outputs.  lora_dropout=0 means no "knobs" are randomly turned off during training.

bias="none" means no extra adjustments to the model's biases. use_gradient_checkpointing="unsloth" is a memory-saving trick for training. random_state=42 ensures we get the same results if we run this again.  use_rslora=False and loftq_config=None are more advanced settings that are turned off here.

In [3]:
! pip uninstall unsloth

^C


In [ ]:
! pip install "unsloth @ git+https://github.com/unslothai/unsloth.git"

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

NameError: name 'FastLanguageModel' is not defined

## Load Dataset

The vicgalle/alpaca-gpt4 dataset is a collection of 52,000 instruction-following instances designed to fine-tune language models (LLMs). It was created by Vic Galie and is available on Hugging Face.

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files = "./medquad.csv",
    split = "train",
)
print(dataset.column_names)

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files = "./medquad.csv",
    split = "train",
)
print(dataset.column_names)

In [ ]:
dataset[0]

In [ ]:
dataset = dataset.map(lambda row: {
    "question": row["question"],
    "answer": f"{row['answer']}\n\nCategory: {row['focus_area']}",
    "focus_area": row["focus_area"]  # Keeping it for reference, can be removed if not needed
})

Let's format the dataset in a way suitable for conversational AI training using the ShareGPT format, which is designed for multi-turn conversations.

In [ ]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt="{question}",
    output_column_name="answer",
    conversation_extension=0,  # Select more to handle longer conversations
)


In [ ]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [ ]:
dataset[0]['conversations']

## Customizable Chat Templates

This code takes a dataset, formats it into a chat-friendly format, and then applies a template so that the AI can understand the instructions and responses correctly.

In [ ]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

In [ ]:

dataset_split = dataset.train_test_split(test_size=0.2, seed=42)

train_data = dataset_split["train"]
val_data = dataset_split["test"]

print(train_data[0]['conversations'], '\n', val_data[0]['conversations'])

## Train the model

Let's define and configure a fine-tuning trainer for a language model using the **SFTTrainer** class from the **trl** library (likely for fine-tuning language models) and transformers' **TrainingArguments**

It uses SFTTrainer from the trl library, which is specifically for Supervised Fine-Tuning.  This means the model will learn from the dataset of instructions and responses you prepared earlier.

It takes the model (the tuned language model), the tokenizer (for breaking down text), and the dataset. dataset_text_field="text" tells it where the actual text is in the dataset.  max_seq_length=2048 limits the length of text sequences the model processes at once. dataset_num_proc=2 uses two processes to prepare the data, and packing=False disables a specific data packing technique.

Then, it configures the training process with TrainingArguments.  per_device_train_batch_size=2 means each training device (like a GPU) will process 2 examples at a time. gradient_accumulation_steps=4 combines the gradients from 4 batches to simulate a larger batch size. warmup_steps=5 gradually increases the learning rate at the beginning. max_steps=20 limits the total training steps. learning_rate=2e-4 sets the learning rate.  fp16 and bf16 control the precision of calculations (using either half-precision or bfloat16 if supported, for faster training). logging_steps=1 logs training progress every step. optim="adamw_8bit" specifies the optimizer. weight_decay=0.01 is a regularization technique. lr_scheduler_type="linear" sets how the learning rate changes over time. seed=3407 ensures reproducibility. output_dir="outputs" specifies where to save the trained model. report_to="none" disables reporting to services like WandB (Weights & Biases).


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

The trainer orchestrates the fine-tuning process by combining the model, tokenizer, dataset, and hyperparameters. It ensures efficient training, taking advantage of mixed precision (fp16 or bf16) and memory-efficient optimizations (e.g., AdamW with 8-bit precision). Additionally, it handles sequence preprocessing, gradient accumulation, and learning rate scheduling.

In [ ]:
trainer_stats = trainer.train()

## Ollama

OLAMA is a powerful tool that enables you to run large language models (LLMs) directly on your own computer (laptop or desktop).


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

Let's save this model.

In [ ]:
model.save_pretrained_gguf("model", tokenizer)

# Start the OLLAMA Server

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3)
print(tokenizer._ollama_modelfile)

This command registers the fine-tuned model (deepseek_finetuned_model) with Ollama. Once registered:

- The model can be run locally using Ollama.
- It becomes accessible for further tasks, such as querying, evaluating, or deploying in specific applications.
- Ollama ensures the model is formatted and stored correctly for efficient usage

In [ ]:
!ollama create deepseek_finetuned_model -f ./model/Modelfile

In [ ]:
!pip install ollama

In [ ]:
import ollama

response = ollama.chat(model="deepseek_finetuned_model",
            messages=[{ "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8,"
            },
                      ])

print(response.message.content)

1, 1, 2, 3, 5, 8, 13, 21

In [ ]:
from IPython.display import Markdown
import ollama

response = ollama.chat(model="deepseek_finetuned_model",
                       messages=[{"role": "user",
                                  "content": "How to add chart to a document?"},
                      ])

Markdown(response.message.content)

To add a chart to a document, follow these steps:

- Insert a Table: Start by inserting a table into the document. You can do this using the 'Table' tool in most word processors.
- Insert Data: Add data into the table. Ensure that your data is properly formatted and organised before adding the chart.
- Choose a Chart Type: Select the type of chart you want to create from the available options (e.g., bar chart, pie chart, line graph, etc.).
- Edit the Chart Data: Add the necessary data points and formatting to the chart using the chart editor that appears once the chart is selected.
- Format the Table and Chart Together: Make sure the table and chart work well together by adjusting alignment, spacing, and other design elements as needed.

For more detailed instructions, you may want to consult a guide or use a tool such as Microsoft Word's chart features.